# **Parking lot generation**

This notebook is used to create the basic simulation. Basic simulation consists of:
- randomized network
- with randomized parking lots (capacity for 400 vehicles)
- 400 random vehicle trips
- which terminates in parking lots
- parking rerouters to everyone find a parking lot

**Prerequisites:**
1. Install [Eclipse SUMO](https://www.eclipse.org/sumo/)
2. Set the SUMO_HOME variable

In [35]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import numpy as np
import os

In [36]:
SUMO_HOME = "/usr/share/sumo"

## **Generating a random SUMO network**

In [37]:
os.system("netgenerate --rand -o ../network.net.xml --rand.iterations=30")

0

## **Creating random trips for the SUMO simulation**

In [38]:
os.system('''python3 %s/tools/randomTrips.py -n ../network.net.xml -o flows.xml --begin 0 --end 1 --flows 200 --jtrrouter --trip-attributes 'departPos="random" departSpeed="max"' '''%SUMO_HOME)

0

In [39]:
os.system("jtrrouter -c flow_gen.jtrrcfg --vtype-output ../vtype.xml")

0

## **Generating parking lots**

### Reading SUMO network

In [40]:
tree = ET.parse('../network.net.xml')
root = tree.getroot()

In [41]:
edge_ids = []
lengths = []
for neighbor in root.findall('edge'):
    if neighbor.attrib["id"][0] != ":":
        edge_ids.append(neighbor.attrib["id"])
        for lane in neighbor.findall("lane"):
            lengths.append(lane.attrib["length"])

In [42]:
edge_ids = np.array(edge_ids)
lengths = np.array(lengths)

### Writing parking lots

In [43]:
num_cars = 200 #how many cars are in the simulation
parking_lot_length = 6 #how long a parking lot shall be, given in meters
save_to = "../parkings.add.xml" #where to save parking description file

In [44]:
#creating the corresponding xml document:
write_root = minidom.Document()

xml = write_root.createElement("additionals")
write_root.appendChild(xml)

<DOM Element: additionals at 0x7f3275dd76d0>

In [45]:
#selecting edges for the parking lots:
generated_lots = 0
gen_ids = []
#generate at least enough parking lots for the vehicles:
while generated_lots < num_cars:
    i = np.random.choice(a=range(0, len(edge_ids)), size=1)[0]
    if not("pl%s"%edge_ids[i] in gen_ids): #only unique parking lots are generated:
        park_area = write_root.createElement("parkingArea")
        park_area.setAttribute("id", "pl%s"%edge_ids[i])
        gen_ids.append("pl%s"%edge_ids[i])
        park_area.setAttribute("lane", "%s_0"%edge_ids[i])
        park_area.setAttribute("startPos", "0")
        park_area.setAttribute("endPos", "%f"%float(lengths[i]))
        park_area.setAttribute("roadsideCapacity", "%d"%(float(lengths[i])//parking_lot_length))
        generated_lots += float(lengths[i])//parking_lot_length
        xml.appendChild(park_area)

In [46]:
xml_str = write_root.toprettyxml(indent ="\t") 
with open(save_to, "w") as f:
    f.write(xml_str)

# **Assigning vehicles to parking lots**

For convenience, we will generate `incomplete routes` in case a vehicle does not pass any parking lot. (SUMO can handle this case.)

In [47]:
save_to_route = "../parking_routes.rou.xml" #file into save the created parking routes

In [48]:
route_doc = minidom.parse('routes.xml') #file where routes are stored without stops at parking lots

In [49]:
for veh in route_doc.getElementsByTagName('vehicle'):
    route = veh.getElementsByTagName("route")
    route = route[0].getAttribute("edges").split(" ")
    route = ["pl"+x for x in route]
    
    i = 0
    while i<len(route) and not(route[-i] in gen_ids): i += 1
    if i>=len(route):
        #no parking along the route
        #then we add one:
        route = veh.getElementsByTagName("route")
        i = np.random.choice(a=range(0, len(gen_ids)), size=1)[0]
        route[0].setAttribute("edges", route[0].getAttribute("edges")+" "+gen_ids[i].split("pl")[-1])
    else:
        #last edge with parking lot is used:
        i = gen_ids.index(route[-i])
    #adding a stop at the selected parking lot:
    stop = route_doc.createElement("stop")
    stop.setAttribute("parkingArea", gen_ids[i])
    stop.setAttribute("duration", "86400")
    veh.setAttribute("reroute", "true")
    veh.appendChild(stop)

In [50]:
xml_str = route_doc.toprettyxml(indent ="\t") 
with open(save_to_route, "w") as f:
    f.write(xml_str)

## **Generating ParkingAreaRerouters**

In [51]:
os.system("python3 %s/tools/generateParkingAreaRerouters.py -n ../network.net.xml -a ../parkings.add.xml --max-distance-alternatives 10000 --max-distance-visibility-true 1000 -o ../parking_rerouters.add.xml"%SUMO_HOME)

0